### Multi-LLM Agentic Sales Agent

This notebook details the following:
1. Different models: Demonstrates how to integrate and use various language models (DeepSeek, Gemini, Llama3.3,  Claude, Amazon Bedrock Nova) within the agentic framework.
2. Structured Outputs: Shows how to define and enforce structured outputs for agents using Pydantic BaseModel.
3. Guardrails: Implements an input guardrail to prevent sensitive information (like personal names) from being processed.

In [ ]:
# Importing necessary components from Strands SDK and other libraries.
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel
from agents.extensions.models.litellm_model import LitellmModel

In [ ]:
# Load environment variables from a .env file. `override=True` will overwrite existing environment variables.
load_dotenv(override=True)

In [ ]:
# Retrieve API keys from environment variables.
amazon_bedrock_api_key = os.getenv('AMAZON_BEDROCK_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

# Print the status of each API key to confirm they are loaded.
if amazon_bedrock_api_key:
    print(f"Amazon Bedrock API Key exists and begins {amazon_bedrock_api_key[:8]}")
else:
    print("Amazon Bedrock API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
# Define different instruction sets for various sales agent personas.
# These instructions guide the behavior and tone of the generated emails.

# Professional sales agent instructions
professional_instructions = "You are a professional sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

# Engaging sales agent instructions
engaging_instructions = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

# Concise sales agent instructions
concise_instructions = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

- It's easy to use any models with OpenAI compatible endpoints

In [ ]:
# Define base URLs for different OpenAI-compatible API endpoints.
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [ ]:
# Initialize AsyncOpenAI clients for each service.
# These clients will be used to interact with the respective LLM APIs.
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

# Create OpenAIChatCompletionsModel instances for each model.
# These models abstract the API interactions and provide a consistent interface for the Agents SDK.
deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [ ]:
# Initialize Agent instances for each sales persona, assigning them a specific model and instructions.
professional_sales_agent = Agent(name="DeepSeek Sales Agent", 
                                instructions=professional_instructions, 
                                model=deepseek_model)
engaging_sales_agent =  Agent(name="Gemini Sales Agent", 
                                instructions=engaging_instructions, 
                                model=gemini_model)
concise_sales_agent  = Agent(name="Llama3.3 Sales Agent",
                                instructions=concise_instructions, 
                                model=llama3_3_model)

In [ ]:
# Define a common description for the sales email writing tools.
description = "Write a cold sales email"

# Convert each sales agent into a tool.
# This allows a higher-level agent (e.g., Sales Manager) to invoke these agents as functions.
professional_tool = professional_sales_agent.as_tool(tool_name="professional_sales_agent", 
                                                    tool_description=description)
engaging_tool = engaging_sales_agent.as_tool(tool_name="engaging_sales_agent", 
                                            tool_description=description)
concise_tool = concise_sales_agent.as_tool(tool_name="concise_sales_agent", 
                                            tool_description=description)

In [ ]:
# Define a function tool to send HTML emails using SendGrid.
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    print("[Tool Call] Send html tool engaged")
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("iankisali@gmail.com")
    to_email = To("iankisali295@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print("[Tool Call] Send html tool completed")
    return {"status": "success"}

In [ ]:
# Define instructions for agents responsible for writing email subjects and converting text to HTML.

# subject writer instructions
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

# html body writer instructions
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

# Initialize an Agent for writing email subjects, using a LiteLLMModel for Anthropic Claude.
subject_writer = Agent(name="Email subject writer", 
                        instructions=subject_instructions, 
                        model=LitellmModel(model="anthropic.claude-3-5-sonnet-20240620-v1:0", api_key=amazon_bedrock_api_key))

# Convert the subject writer agent into a tool.
subject_tool = subject_writer.as_tool(tool_name="subject_writer_tool", 
                        tool_description="Write a subject for a cold sales email")

# Initialize an Agent for converting email bodies to HTML, also using Anthropic Claude.
html_converter = Agent(name="HTML email body converter", 
                        instructions=html_instructions, 
                        model=LitellmModel(model="anthropic.claude-3-5-sonnet-20240620-v1:0", api_key=amazon_bedrock_api_key))

# Convert the HTML converter agent into a tool.
html_tool = html_converter.as_tool(tool_name="html_converter_tool",
                        tool_description="Convert a text email body to an HTML email body")

In [ ]:
# Group all email-related tools together.
email_tools = [subject_tool, html_tool, send_html_email]

In [ ]:
# Display the list of email tools (for verification).
email_tools

In [ ]:
# Define instructions for the Email Manager agent.
email_formater_instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

# Initialize the Email Manager agent. This agent orchestrates the email formatting and sending process.
emailer_manager_agent = Agent(
    name="Email Manager",
    instructions=email_formater_instructions,
    tools=email_tools,
    model=LitellmModel(model="amazon.nova-pro-v1:0", api_key=amazon_bedrock_api_key),
    handoff_description="Convert an email to HTML and send it")

In [ ]:
# Group all sales agent tools.
tools = [professional_tool, concise_tool, engaging_tool]
# Define handoffs for the Sales Manager, which is the Email Manager agent.
handoffs = [emailer_manager_agent]

In [ ]:
# Define instructions for the Sales Manager agent.
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

# Initialize the Sales Manager agent. This is the top-level agent that decides which sales agent to use and then hands off to the Email Manager.
sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=LitellmModel(model="amazon.nova-pro-v1:0", api_key=amazon_bedrock_api_key))

# Define the initial message for the sales manager.
message = "Send out a cold sales email addressed to Dear Safaricom CEO from Ian Kisali"

# Run the sales process and trace its execution.
# The `trace` context manager helps in visualizing the agent's decision-making process.
with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

- Check out the trace on OpenAI SDK Developer platform:

https://platform.openai.com/traces

In [ ]:
# Define a Pydantic BaseModel for structured output from the name check guardrail.
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

# Initialize a guardrail agent. Its purpose is to check if a personal name is present in the input message.
guardrail_agent = Agent( 
    name="Name checker agent",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model=LitellmModel(model="anthropic.claude-3-5-sonnet-20240620-v1:0", api_key=amazon_bedrock_api_key)
)

In [ ]:
# Define an input guardrail function.
# This function uses the `guardrail_agent` to check for names and triggers a "tripwire" if a name is found.
# If a name is found, `tripwire_triggered` is set to True, which will halt the execution.
@input_guardrail
async def name_guardrail(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
# Initialize a "careful" sales manager agent that includes the `name_guardrail`.
# This agent will prevent execution if a personal name is detected in the initial message.
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_manager_agent],
    model=deepseek_model,
    input_guardrails=[name_guardrail]
    )

# Attempt to run with a message containing a personal name.
# This is expected to trigger the guardrail and raise an error
message = "Send out a cold sales email addressed to Dear Safaricom CEO from Ian Kisali"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

- Check out the trace:

https://platform.openai.com/traces

In [ ]:
# Attempt to run with a message that does not contain a personal name.
# This should proceed without triggering the guardrail.
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)